# Dependencies

In [ ]:
import pandas as pd
import numpy as np
from itertools import cycle
from scipy.stats import zscore
from datetime import datetime, timedelta

# Load and process data

In [ ]:
kitchen = pd.read_csv('COVID_kitchen.csv').iloc[:,1:]
kitchen['start_date'] = pd.to_datetime(kitchen['start_date'])
kitchen = kitchen.assign(Event=1)
kitchen['activity'] = kitchen.activity.astype(str).replace({'oven-use':'Appliance', 'Kettle':'Appliance', 'fridge door':'Fridge Door'})
kitchen

# Generate datasets for statistical analyses

## Pandemic and occupancy datasets

In [ ]:
COVID = kitchen.copy()
COVID['start_date'] = COVID['start_date'].dt.date
COVID['start_date'] = pd.to_datetime(COVID['start_date'])
COVID = pd.pivot_table(COVID, index=['COVID','Occupancy','household','start_date'], columns='activity', values='Event', aggfunc=np.sum, fill_value=0).reset_index()
COVID = pd.melt(COVID, id_vars=['COVID','Occupancy','household','start_date'], value_vars=['Appliance','Fridge Door','Kitchen'], var_name='activity', value_name='Event')
COVID

In [ ]:
COVID_wo = pd.pivot_table(COVID, index=['COVID', 'household'], columns='activity', values='Event', aggfunc=np.mean, fill_value=0)
COVID_wo['daily_sum'] = COVID_wo.sum(axis=1)
COVID_wo = COVID_wo[['daily_sum']].reset_index() 

COVID_occupancy = pd.pivot_table(COVID, index=['COVID', 'household', 'Occupancy'], columns='activity', values='Event', aggfunc=np.mean, fill_value=0)
COVID_occupancy['daily_sum'] = COVID_occupancy.sum(axis=1)
COVID_occupancy = COVID_occupancy[['daily_sum']].reset_index()

In [ ]:
Period = kitchen.copy()
Period['start_date'] = Period['start_date'].dt.date
Period['start_date'] = pd.to_datetime(Period['start_date'])
Period = pd.pivot_table(Period, index=['Period','Occupancy','household','start_date'], columns='activity', values='Event', aggfunc=np.sum, fill_value=0).reset_index()
Period = pd.melt(Period, id_vars=['Period','Occupancy','household','start_date'], value_vars=['Appliance','Fridge Door','Kitchen'], var_name='activity', value_name='Event')
Period

In [ ]:
Period_wo = pd.pivot_table(Period, index=['Period', 'household'], columns='activity', values='Event', aggfunc=np.mean, fill_value=0)
Period_wo['daily_sum'] = Period_wo.sum(axis=1)
Period_wo = Period_wo[['daily_sum']].reset_index() 

Period_occupancy = pd.pivot_table(Period, index=['Period', 'household', 'Occupancy'], columns='activity', values='Event', aggfunc=np.mean, fill_value=0)
Period_occupancy['daily_sum'] = Period_occupancy.sum(axis=1)
Period_occupancy = Period_occupancy[['daily_sum']].reset_index()

In [ ]:
COVID_wo.to_csv('COVID_wo.csv')
COVID_occupancy.to_csv('COVID_occupancy.csv')
Period_wo.to_csv('Period_wo.csv')
Period_occupancy.to_csv('Period_occupancy.csv')

## Pandemic and time of day datasets

In [ ]:
COVID_tod = kitchen.set_index('start_date').sort_index().groupby(['COVID','Occupancy','household','activity']).resample('6h').Event.sum().reset_index()

COVID_tod = pd.pivot_table(COVID_tod, index=['COVID','Occupancy','household','start_date'], columns='activity', values='Event', fill_value=0).reset_index()
COVID_tod['time_of_day'] = COVID_tod['start_date'].dt.hour
COVID_tod['start_date'] = COVID_tod['start_date'].dt.date
COVID_tod = pd.melt(COVID_tod, id_vars=['COVID','Occupancy','household','start_date','time_of_day'], value_vars=['Appliance','Fridge Door','Kitchen'], var_name='activity', value_name='Event')
COVID_tod = pd.pivot_table(COVID_tod, index=['COVID', 'household', 'time_of_day'], columns='activity', values='Event', aggfunc=np.mean, fill_value=0)
COVID_tod['daily_sum'] = COVID_tod.sum(axis=1)
COVID_tod = COVID_tod[['daily_sum']].reset_index()

time_period = cycle(['night','morning','afternoon','evening'])
COVID_tod['time_of_day'] = [next(time_period) for count in range(COVID_tod.shape[0])]

COVID_tod_z = COVID_tod.groupby("time_of_day").transform(lambda x : zscore(x, ddof=1))
COVID_tod['daily_sum'] = COVID_tod_z['daily_sum']
COVID_tod 

In [ ]:
Period_tod = kitchen.set_index('start_date').sort_index().groupby(['Period','Occupancy','household','activity']).resample('6h').Event.sum().reset_index()

Period_tod = pd.pivot_table(Period_tod, index=['Period','Occupancy','household','start_date'], columns='activity', values='Event', fill_value=0).reset_index()
Period_tod['time_of_day'] = Period_tod['start_date'].dt.hour
Period_tod['start_date'] = Period_tod['start_date'].dt.date
Period_tod = pd.melt(Period_tod, id_vars=['Period','Occupancy','household','start_date','time_of_day'], value_vars=['Appliance','Fridge Door','Kitchen'], var_name='activity', value_name='Event')
Period_tod = pd.pivot_table(Period_tod, index=['Period', 'household', 'time_of_day'], columns='activity', values='Event', aggfunc=np.mean, fill_value=0)
Period_tod['daily_sum'] = Period_tod.sum(axis=1)
Period_tod = Period_tod[['daily_sum']].reset_index()

time_period = cycle(['night','morning','afternoon','evening'])
Period_tod['time_of_day'] = [next(time_period) for count in range(Period_tod.shape[0])]

Period_tod_z = Period_tod.groupby("time_of_day").transform(lambda x : zscore(x, ddof=1))
Period_tod['daily_sum'] = Period_tod_z['daily_sum']
Period_tod 

In [ ]:
COVID_tod.to_csv('COVID_tod.csv')
Period_tod.to_csv('Period_tod.csv')

# Run statistical analyses in RStudio

In a new session, import the 6 datasets derived using the code above.

Next, in the Console, load lmerTest package using the following line of code:

    library(lmerTest)

Fit each model with the respective line of code:

    lmm1 <- lmer(daily_sum ~ COVID + (1|household), data=COVID_wo, REML=FALSE)

    lmm2 <- lmer(daily_sum ~ COVID * Occupancy + (1|household), data=COVID_occupancy, REML=FALSE)

    lmm3 <- lmer(daily_sum ~ Period + (1|household), data=Period_wo, REML=FALSE)

    lmm4 <- lmer(daily_sum ~ Period * Occupancy + (1|household), data=Period_occupancy, REML=FALSE)

    lmm5 <- lmer(daily_sum ~ COVID * time_of_day + (1|household), data=COVID_tod, REML=FALSE)

    lmm6 <- lmer(daily_sum ~ Period * time_of_day + (1|household), data=Period_tod, REML=FALSE)

Now, print summary including coefficient table with p-values for t-statistics using Satterthwaite's method for denominator degrees of freedom:

    print(summary(model_name)) # where model name is lmm1, lmm2...

Finally, print Type III anova table with p-values for F-tests based on Satterthwaite's method:

    print(anova(model_name)) # where model name is lmm1, lmm2...
